# 네이버 API에서 키워드를 입력받아 블로그, 뉴스, 책 데이터 수집
# 각각의 카테고리별로 csv 파일로 저장
# 파일명에는 수집날짜가 자동으로 파일명에 포함되도록
# 반복되는 부분은 함수를 이용해서 중복 줄이기

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import re
from datetime import datetime
load_dotenv(dotenv_path="./data/.env_naver")

False

In [6]:
def text_clean(text):
    # html 태그를 없애는 정규표현식
    result = re.sub(r"</?[^>]+>", "", text)
    # 한글, 영문, 숫자 외의 모든 문자 제거 후 공백으로 변환
    result = re.sub(r"[^가-힣a-zA-Z0-9]", " ", result)
    result = result.replace("  ", " ").replace("  ", " ").replace("  ", " ")
    return result

# 1.페이지 수집

In [3]:
keyword = input("검색할 키워드를 입력해 주세요")

검색할 키워드를 입력해 주세요부동산


In [2]:
import json
user_id = os.getenv("user_id")
user_secret = os.getenv("user_secret")
url = f"https://openapi.naver.com/v1/search/news"
payload = dict(query=keyword, display=100, start=1, sort="date")
headers = {"X-Naver-Client-Id" : user_id, "X-Naver-Client-Secret" : user_secret}
r = requests.get(url, params=payload, headers=headers)
response = r.json()

total_page = response['total'] // 100 + 1

result = {}
for item in response['items']:
#     print(item)
    for key, value in item.items():
        if key in ['title', 'description']:
            result.setdefault(key, []).append(text_clean(value))
        else:
            result.setdefault(key, []).append(value)
df = pd.DataFrame(result)
df

NameError: name 'keyword' is not defined

In [8]:
keyword = input("검색할 키워드를 입력해 주세요")


blog_data = []
news_data = []
book_data = []

category = ['blog', 'news', 'book']

for cate in category:
    page_num = 1
    total_page = 1
    start_num = 1
    while page_num <= total_page:
        print(cate, page_num, total_page, end="\r")
        user_id = os.getenv("user_id")
        user_secret = os.getenv("user_secret")
        url = f"https://openapi.naver.com/v1/search/{cate}"
        payload = dict(query=keyword, display=100, start=start_num, sort="date")
        headers = {"X-Naver-Client-Id" : user_id, "X-Naver-Client-Secret" : user_secret}
        r = requests.get(url, params=payload, headers=headers)
        response = r.json()

        result = {}
        for item in response['items']:

            for key, value in item.items():
                if key in ['title', 'description']:
                    result.setdefault(key, []).append(text_clean(value))
                else:
                    result.setdefault(key, []).append(value)
        
        if cate == 'blog':
            blog_data.append(pd.DataFrame(result))
        elif cate == 'news':
            news_data.append(pd.DataFrame(result))
        elif cate == 'book':
            book_data.append(pd.DataFrame(result))

        total_page = response['total'] // 100 + 1
        if total_page > 11:
            total_page = 11
        else:
            total_page = response['total'] // 100 + 1


        page_num += 1

        if start_num < 901:
            start_num += 100
        elif start_num > 900:
            start_num += 99

            
            
blog_df = pd.concat(blog_data)
news_df = pd.concat(news_data)
book_df = pd.concat(book_data)

blog_df = blog_df.reset_index(drop=True)
news_df = news_df.reset_index(drop=True)
book_df = book_df.reset_index(drop=True)

display(blog_df)
display(news_df)
display(book_df)

검색할 키워드를 입력해 주세요부동산


KeyError: 'items'

In [9]:
news_data

[]

In [10]:
news_df = pd.concat(news_data)

ValueError: No objects to concatenate

In [11]:
news_df

NameError: name 'news_df' is not defined

# 코드를 함수화

In [12]:
def naver_api(keyword, cate):
    page_num = 1
    total_page = 1
    start_num = 1
    
    all_data = []
    while page_num <= total_page:
        print(cate, page_num, total_page, end="\r")
        user_id = os.getenv("user_id")
        user_secret = os.getenv("user_secret")
        url = f"https://openapi.naver.com/v1/search/{cate}"
        payload = dict(query=keyword, display=100, start=start_num, sort="date")
        headers = {"X-Naver-Client-Id" : user_id, "X-Naver-Client-Secret" : user_secret}
        r = requests.get(url, params=payload, headers=headers)
        response = r.json()

        result = {}
        for item in response['items']:

            for key, value in item.items():
                if key in ['title', 'description']:
                    result.setdefault(key, []).append(text_clean(value))
                else:
                    result.setdefault(key, []).append(value)
        
        all_data.append(pd.DataFrame(result))
        
        total_page = response['total'] // 100 + 1
        if total_page > 11:
            total_page = 11
        else:
            total_page = response['total'] // 100 + 1


        page_num += 1

        if start_num < 901:
            start_num += 100
        elif start_num > 900:
            start_num += 99

    
    result_df = pd.concat(all_data)
    result_df = result_df.reset_index(drop=True)
    return result_df

In [13]:
today = datetime.today()
today

datetime.datetime(2025, 10, 21, 11, 26, 9, 469856)

In [14]:
date = f"{today.year}{today.month:02d}{today.day:02d}"
date

'20251021'

In [15]:
keyword = input("검색어를 입력하세요")
date = f"{today.year}{today.month:02d}{today.day:02d}"
for cate in category:
    result = naver_api(keyword, cate)
    result.to_csv(f"./data/{keyword}_{cate}_{date}.csv", index=False, encoding="utf-8-sig")

검색어를 입력하세요핀테크


KeyError: 'items'